# Import Library & Download Resource

In [83]:
import pandas as pd
import nltk
import re
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import ast
import seaborn as sns
import plotly.express as px

In [2]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/muhakbarhamid21/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/muhakbarhamid21/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/muhakbarhamid21/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from transformers import AutoTokenizer, AutoModel
from sklearn.decomposition import PCA

# Import Dataset Text

In [4]:
df = pd.read_csv('../../datasets/final-data/data_text_batik.csv')

In [5]:
df.iloc[11]['Deskripsi']

'Batik Megamendung memiliki motif awan bergradasi dengan warna biru, putih, atau merah. Melambangkan kesejukan dan ketenangan, batik ini merupakan warisan budaya dari Cirebon, Jawa Barat.'

In [6]:
df

,Nama,Deskripsi,Path Folder,Path Gambar
0,Batik Bali,Batik Bali memiliki motif rumit yang terinspir...,batik-bali,batik-bali.jpg
1,Batik Betawi,Batik Betawi memiliki motif geometris sederhan...,batik-betawi,batik-betawi.jpg
2,Batik Celup,Batik Celup memiliki pola abstrak dengan grada...,batik-celup,batik-celup.jpg
3,Batik Cendrawasih,Batik Cendrawasih menampilkan motif burung cen...,batik-cendrawasih,batik-cendrawasih.jpg
4,Batik Ceplok,Batik Ceplok memiliki pola geometris berulang ...,batik-ceplok,batik-ceplok.jpg
5,Batik Ciamis,Batik Ciamis memiliki motif sederhana dengan p...,batik-ciamis,batik-ciamis.jpg
6,Batik Garutan,"Batik Garutan memiliki motif bunga, daun, atau...",batik-garutan,batik-garutan.jpg
7,Batik Gentongan,Batik Gentongan memiliki motif bunga besar den...,batik-gentongan,batik-gentongan.jpg
8,Batik Kawung,Batik Kawung memiliki motif lingkaran geometri...,batik-kawung,batik-kawung.jpg
9,Batik Keraton,Batik Keraton memiliki motif klasik seperti Pa...,batik-keraton,batik-keraton.jpg


# Pre-processing

## Case Folding

In [7]:
df['Deskripsi_casefolding'] = df['Deskripsi'].str.lower()

In [8]:
df.iloc[11]['Deskripsi_casefolding']

'batik megamendung memiliki motif awan bergradasi dengan warna biru, putih, atau merah. melambangkan kesejukan dan ketenangan, batik ini merupakan warisan budaya dari cirebon, jawa barat.'

In [9]:
df

,Nama,Deskripsi,Path Folder,Path Gambar,Deskripsi_casefolding
0,Batik Bali,Batik Bali memiliki motif rumit yang terinspir...,batik-bali,batik-bali.jpg,batik bali memiliki motif rumit yang terinspir...
1,Batik Betawi,Batik Betawi memiliki motif geometris sederhan...,batik-betawi,batik-betawi.jpg,batik betawi memiliki motif geometris sederhan...
2,Batik Celup,Batik Celup memiliki pola abstrak dengan grada...,batik-celup,batik-celup.jpg,batik celup memiliki pola abstrak dengan grada...
3,Batik Cendrawasih,Batik Cendrawasih menampilkan motif burung cen...,batik-cendrawasih,batik-cendrawasih.jpg,batik cendrawasih menampilkan motif burung cen...
4,Batik Ceplok,Batik Ceplok memiliki pola geometris berulang ...,batik-ceplok,batik-ceplok.jpg,batik ceplok memiliki pola geometris berulang ...
5,Batik Ciamis,Batik Ciamis memiliki motif sederhana dengan p...,batik-ciamis,batik-ciamis.jpg,batik ciamis memiliki motif sederhana dengan p...
6,Batik Garutan,"Batik Garutan memiliki motif bunga, daun, atau...",batik-garutan,batik-garutan.jpg,"batik garutan memiliki motif bunga, daun, atau..."
7,Batik Gentongan,Batik Gentongan memiliki motif bunga besar den...,batik-gentongan,batik-gentongan.jpg,batik gentongan memiliki motif bunga besar den...
8,Batik Kawung,Batik Kawung memiliki motif lingkaran geometri...,batik-kawung,batik-kawung.jpg,batik kawung memiliki motif lingkaran geometri...
9,Batik Keraton,Batik Keraton memiliki motif klasik seperti Pa...,batik-keraton,batik-keraton.jpg,batik keraton memiliki motif klasik seperti pa...


## Text Cleaning

Removes all non-letter characters (a-z) and spaces.

In [10]:
df['Deskripsi_clean'] = df['Deskripsi_casefolding'].apply(
    lambda text: re.sub(r'[^a-z\s]', '', text)
)

In [11]:
df.iloc[11]['Deskripsi_clean']

'batik megamendung memiliki motif awan bergradasi dengan warna biru putih atau merah melambangkan kesejukan dan ketenangan batik ini merupakan warisan budaya dari cirebon jawa barat'

In [12]:
df

,Nama,Deskripsi,Path Folder,Path Gambar,Deskripsi_casefolding,Deskripsi_clean
0,Batik Bali,Batik Bali memiliki motif rumit yang terinspir...,batik-bali,batik-bali.jpg,batik bali memiliki motif rumit yang terinspir...,batik bali memiliki motif rumit yang terinspir...
1,Batik Betawi,Batik Betawi memiliki motif geometris sederhan...,batik-betawi,batik-betawi.jpg,batik betawi memiliki motif geometris sederhan...,batik betawi memiliki motif geometris sederhan...
2,Batik Celup,Batik Celup memiliki pola abstrak dengan grada...,batik-celup,batik-celup.jpg,batik celup memiliki pola abstrak dengan grada...,batik celup memiliki pola abstrak dengan grada...
3,Batik Cendrawasih,Batik Cendrawasih menampilkan motif burung cen...,batik-cendrawasih,batik-cendrawasih.jpg,batik cendrawasih menampilkan motif burung cen...,batik cendrawasih menampilkan motif burung cen...
4,Batik Ceplok,Batik Ceplok memiliki pola geometris berulang ...,batik-ceplok,batik-ceplok.jpg,batik ceplok memiliki pola geometris berulang ...,batik ceplok memiliki pola geometris berulang ...
5,Batik Ciamis,Batik Ciamis memiliki motif sederhana dengan p...,batik-ciamis,batik-ciamis.jpg,batik ciamis memiliki motif sederhana dengan p...,batik ciamis memiliki motif sederhana dengan p...
6,Batik Garutan,"Batik Garutan memiliki motif bunga, daun, atau...",batik-garutan,batik-garutan.jpg,"batik garutan memiliki motif bunga, daun, atau...",batik garutan memiliki motif bunga daun atau b...
7,Batik Gentongan,Batik Gentongan memiliki motif bunga besar den...,batik-gentongan,batik-gentongan.jpg,batik gentongan memiliki motif bunga besar den...,batik gentongan memiliki motif bunga besar den...
8,Batik Kawung,Batik Kawung memiliki motif lingkaran geometri...,batik-kawung,batik-kawung.jpg,batik kawung memiliki motif lingkaran geometri...,batik kawung memiliki motif lingkaran geometri...
9,Batik Keraton,Batik Keraton memiliki motif klasik seperti Pa...,batik-keraton,batik-keraton.jpg,batik keraton memiliki motif klasik seperti pa...,batik keraton memiliki motif klasik seperti pa...


## Tokenization

In [15]:
df['tokenized'] = df['Deskripsi_clean'].apply(lambda text: text.split())

In [16]:
df.iloc[11]['tokenized']

['batik',
 'megamendung',
 'memiliki',
 'motif',
 'awan',
 'bergradasi',
 'dengan',
 'warna',
 'biru',
 'putih',
 'atau',
 'merah',
 'melambangkan',
 'kesejukan',
 'dan',
 'ketenangan',
 'batik',
 'ini',
 'merupakan',
 'warisan',
 'budaya',
 'dari',
 'cirebon',
 'jawa',
 'barat']

In [17]:
df

,Nama,Deskripsi,Path Folder,Path Gambar,Deskripsi_casefolding,Deskripsi_clean,tokenized
0,Batik Bali,Batik Bali memiliki motif rumit yang terinspir...,batik-bali,batik-bali.jpg,batik bali memiliki motif rumit yang terinspir...,batik bali memiliki motif rumit yang terinspir...,"[batik, bali, memiliki, motif, rumit, yang, te..."
1,Batik Betawi,Batik Betawi memiliki motif geometris sederhan...,batik-betawi,batik-betawi.jpg,batik betawi memiliki motif geometris sederhan...,batik betawi memiliki motif geometris sederhan...,"[batik, betawi, memiliki, motif, geometris, se..."
2,Batik Celup,Batik Celup memiliki pola abstrak dengan grada...,batik-celup,batik-celup.jpg,batik celup memiliki pola abstrak dengan grada...,batik celup memiliki pola abstrak dengan grada...,"[batik, celup, memiliki, pola, abstrak, dengan..."
3,Batik Cendrawasih,Batik Cendrawasih menampilkan motif burung cen...,batik-cendrawasih,batik-cendrawasih.jpg,batik cendrawasih menampilkan motif burung cen...,batik cendrawasih menampilkan motif burung cen...,"[batik, cendrawasih, menampilkan, motif, burun..."
4,Batik Ceplok,Batik Ceplok memiliki pola geometris berulang ...,batik-ceplok,batik-ceplok.jpg,batik ceplok memiliki pola geometris berulang ...,batik ceplok memiliki pola geometris berulang ...,"[batik, ceplok, memiliki, pola, geometris, ber..."
5,Batik Ciamis,Batik Ciamis memiliki motif sederhana dengan p...,batik-ciamis,batik-ciamis.jpg,batik ciamis memiliki motif sederhana dengan p...,batik ciamis memiliki motif sederhana dengan p...,"[batik, ciamis, memiliki, motif, sederhana, de..."
6,Batik Garutan,"Batik Garutan memiliki motif bunga, daun, atau...",batik-garutan,batik-garutan.jpg,"batik garutan memiliki motif bunga, daun, atau...",batik garutan memiliki motif bunga daun atau b...,"[batik, garutan, memiliki, motif, bunga, daun,..."
7,Batik Gentongan,Batik Gentongan memiliki motif bunga besar den...,batik-gentongan,batik-gentongan.jpg,batik gentongan memiliki motif bunga besar den...,batik gentongan memiliki motif bunga besar den...,"[batik, gentongan, memiliki, motif, bunga, bes..."
8,Batik Kawung,Batik Kawung memiliki motif lingkaran geometri...,batik-kawung,batik-kawung.jpg,batik kawung memiliki motif lingkaran geometri...,batik kawung memiliki motif lingkaran geometri...,"[batik, kawung, memiliki, motif, lingkaran, ge..."
9,Batik Keraton,Batik Keraton memiliki motif klasik seperti Pa...,batik-keraton,batik-keraton.jpg,batik keraton memiliki motif klasik seperti pa...,batik keraton memiliki motif klasik seperti pa...,"[batik, keraton, memiliki, motif, klasik, sepe..."


## Stop Word Removal

In [18]:
stop_words = set(stopwords.words('indonesian'))

In [19]:
df['stop_removed'] = df['tokenized'].apply(lambda tokens: [token for token in tokens if token.lower() not in stop_words])

In [20]:
df.iloc[11]['stop_removed']

['batik',
 'megamendung',
 'memiliki',
 'motif',
 'awan',
 'bergradasi',
 'warna',
 'biru',
 'putih',
 'merah',
 'melambangkan',
 'kesejukan',
 'ketenangan',
 'batik',
 'warisan',
 'budaya',
 'cirebon',
 'jawa',
 'barat']

In [21]:
df

,Nama,Deskripsi,Path Folder,Path Gambar,Deskripsi_casefolding,Deskripsi_clean,tokenized,stop_removed
0,Batik Bali,Batik Bali memiliki motif rumit yang terinspir...,batik-bali,batik-bali.jpg,batik bali memiliki motif rumit yang terinspir...,batik bali memiliki motif rumit yang terinspir...,"[batik, bali, memiliki, motif, rumit, yang, te...","[batik, bali, memiliki, motif, rumit, terinspi..."
1,Batik Betawi,Batik Betawi memiliki motif geometris sederhan...,batik-betawi,batik-betawi.jpg,batik betawi memiliki motif geometris sederhan...,batik betawi memiliki motif geometris sederhan...,"[batik, betawi, memiliki, motif, geometris, se...","[batik, betawi, memiliki, motif, geometris, se..."
2,Batik Celup,Batik Celup memiliki pola abstrak dengan grada...,batik-celup,batik-celup.jpg,batik celup memiliki pola abstrak dengan grada...,batik celup memiliki pola abstrak dengan grada...,"[batik, celup, memiliki, pola, abstrak, dengan...","[batik, celup, memiliki, pola, abstrak, gradas..."
3,Batik Cendrawasih,Batik Cendrawasih menampilkan motif burung cen...,batik-cendrawasih,batik-cendrawasih.jpg,batik cendrawasih menampilkan motif burung cen...,batik cendrawasih menampilkan motif burung cen...,"[batik, cendrawasih, menampilkan, motif, burun...","[batik, cendrawasih, menampilkan, motif, burun..."
4,Batik Ceplok,Batik Ceplok memiliki pola geometris berulang ...,batik-ceplok,batik-ceplok.jpg,batik ceplok memiliki pola geometris berulang ...,batik ceplok memiliki pola geometris berulang ...,"[batik, ceplok, memiliki, pola, geometris, ber...","[batik, ceplok, memiliki, pola, geometris, ber..."
5,Batik Ciamis,Batik Ciamis memiliki motif sederhana dengan p...,batik-ciamis,batik-ciamis.jpg,batik ciamis memiliki motif sederhana dengan p...,batik ciamis memiliki motif sederhana dengan p...,"[batik, ciamis, memiliki, motif, sederhana, de...","[batik, ciamis, memiliki, motif, sederhana, po..."
6,Batik Garutan,"Batik Garutan memiliki motif bunga, daun, atau...",batik-garutan,batik-garutan.jpg,"batik garutan memiliki motif bunga, daun, atau...",batik garutan memiliki motif bunga daun atau b...,"[batik, garutan, memiliki, motif, bunga, daun,...","[batik, garutan, memiliki, motif, bunga, daun,..."
7,Batik Gentongan,Batik Gentongan memiliki motif bunga besar den...,batik-gentongan,batik-gentongan.jpg,batik gentongan memiliki motif bunga besar den...,batik gentongan memiliki motif bunga besar den...,"[batik, gentongan, memiliki, motif, bunga, bes...","[batik, gentongan, memiliki, motif, bunga, war..."
8,Batik Kawung,Batik Kawung memiliki motif lingkaran geometri...,batik-kawung,batik-kawung.jpg,batik kawung memiliki motif lingkaran geometri...,batik kawung memiliki motif lingkaran geometri...,"[batik, kawung, memiliki, motif, lingkaran, ge...","[batik, kawung, memiliki, motif, lingkaran, ge..."
9,Batik Keraton,Batik Keraton memiliki motif klasik seperti Pa...,batik-keraton,batik-keraton.jpg,batik keraton memiliki motif klasik seperti pa...,batik keraton memiliki motif klasik seperti pa...,"[batik, keraton, memiliki, motif, klasik, sepe...","[batik, keraton, memiliki, motif, klasik, para..."


## Lemmatization

In [22]:
lemmatizer = WordNetLemmatizer()

In [23]:
df['lemmatized'] = df['stop_removed'].apply(lambda tokens: [lemmatizer.lemmatize(token) for token in tokens])

In [24]:
df.iloc[11]['lemmatized']

['batik',
 'megamendung',
 'memiliki',
 'motif',
 'awan',
 'bergradasi',
 'warna',
 'biru',
 'putih',
 'merah',
 'melambangkan',
 'kesejukan',
 'ketenangan',
 'batik',
 'warisan',
 'budaya',
 'cirebon',
 'jawa',
 'barat']

In [25]:
df

,Nama,Deskripsi,Path Folder,Path Gambar,Deskripsi_casefolding,Deskripsi_clean,tokenized,stop_removed,lemmatized
0,Batik Bali,Batik Bali memiliki motif rumit yang terinspir...,batik-bali,batik-bali.jpg,batik bali memiliki motif rumit yang terinspir...,batik bali memiliki motif rumit yang terinspir...,"[batik, bali, memiliki, motif, rumit, yang, te...","[batik, bali, memiliki, motif, rumit, terinspi...","[batik, bali, memiliki, motif, rumit, terinspi..."
1,Batik Betawi,Batik Betawi memiliki motif geometris sederhan...,batik-betawi,batik-betawi.jpg,batik betawi memiliki motif geometris sederhan...,batik betawi memiliki motif geometris sederhan...,"[batik, betawi, memiliki, motif, geometris, se...","[batik, betawi, memiliki, motif, geometris, se...","[batik, betawi, memiliki, motif, geometris, se..."
2,Batik Celup,Batik Celup memiliki pola abstrak dengan grada...,batik-celup,batik-celup.jpg,batik celup memiliki pola abstrak dengan grada...,batik celup memiliki pola abstrak dengan grada...,"[batik, celup, memiliki, pola, abstrak, dengan...","[batik, celup, memiliki, pola, abstrak, gradas...","[batik, celup, memiliki, pola, abstrak, gradas..."
3,Batik Cendrawasih,Batik Cendrawasih menampilkan motif burung cen...,batik-cendrawasih,batik-cendrawasih.jpg,batik cendrawasih menampilkan motif burung cen...,batik cendrawasih menampilkan motif burung cen...,"[batik, cendrawasih, menampilkan, motif, burun...","[batik, cendrawasih, menampilkan, motif, burun...","[batik, cendrawasih, menampilkan, motif, burun..."
4,Batik Ceplok,Batik Ceplok memiliki pola geometris berulang ...,batik-ceplok,batik-ceplok.jpg,batik ceplok memiliki pola geometris berulang ...,batik ceplok memiliki pola geometris berulang ...,"[batik, ceplok, memiliki, pola, geometris, ber...","[batik, ceplok, memiliki, pola, geometris, ber...","[batik, ceplok, memiliki, pola, geometris, ber..."
5,Batik Ciamis,Batik Ciamis memiliki motif sederhana dengan p...,batik-ciamis,batik-ciamis.jpg,batik ciamis memiliki motif sederhana dengan p...,batik ciamis memiliki motif sederhana dengan p...,"[batik, ciamis, memiliki, motif, sederhana, de...","[batik, ciamis, memiliki, motif, sederhana, po...","[batik, ciamis, memiliki, motif, sederhana, po..."
6,Batik Garutan,"Batik Garutan memiliki motif bunga, daun, atau...",batik-garutan,batik-garutan.jpg,"batik garutan memiliki motif bunga, daun, atau...",batik garutan memiliki motif bunga daun atau b...,"[batik, garutan, memiliki, motif, bunga, daun,...","[batik, garutan, memiliki, motif, bunga, daun,...","[batik, garutan, memiliki, motif, bunga, daun,..."
7,Batik Gentongan,Batik Gentongan memiliki motif bunga besar den...,batik-gentongan,batik-gentongan.jpg,batik gentongan memiliki motif bunga besar den...,batik gentongan memiliki motif bunga besar den...,"[batik, gentongan, memiliki, motif, bunga, bes...","[batik, gentongan, memiliki, motif, bunga, war...","[batik, gentongan, memiliki, motif, bunga, war..."
8,Batik Kawung,Batik Kawung memiliki motif lingkaran geometri...,batik-kawung,batik-kawung.jpg,batik kawung memiliki motif lingkaran geometri...,batik kawung memiliki motif lingkaran geometri...,"[batik, kawung, memiliki, motif, lingkaran, ge...","[batik, kawung, memiliki, motif, lingkaran, ge...","[batik, kawung, memiliki, motif, lingkaran, ge..."
9,Batik Keraton,Batik Keraton memiliki motif klasik seperti Pa...,batik-keraton,batik-keraton.jpg,batik keraton memiliki motif klasik seperti pa...,batik keraton memiliki motif klasik seperti pa...,"[batik, keraton, memiliki, motif, klasik, sepe...","[batik, keraton, memiliki, motif, klasik, para...","[batik, keraton, memiliki, motif, klasik, para..."


## Detokenization

In [26]:
df['preprocessing'] = df['lemmatized'].apply(lambda tokens: ' '.join(tokens))


In [27]:
df.iloc[11]['preprocessing']

'batik megamendung memiliki motif awan bergradasi warna biru putih merah melambangkan kesejukan ketenangan batik warisan budaya cirebon jawa barat'

In [28]:
df

,Nama,Deskripsi,Path Folder,Path Gambar,Deskripsi_casefolding,Deskripsi_clean,tokenized,stop_removed,lemmatized,preprocessing
0,Batik Bali,Batik Bali memiliki motif rumit yang terinspir...,batik-bali,batik-bali.jpg,batik bali memiliki motif rumit yang terinspir...,batik bali memiliki motif rumit yang terinspir...,"[batik, bali, memiliki, motif, rumit, yang, te...","[batik, bali, memiliki, motif, rumit, terinspi...","[batik, bali, memiliki, motif, rumit, terinspi...",batik bali memiliki motif rumit terinspirasi b...
1,Batik Betawi,Batik Betawi memiliki motif geometris sederhan...,batik-betawi,batik-betawi.jpg,batik betawi memiliki motif geometris sederhan...,batik betawi memiliki motif geometris sederhan...,"[batik, betawi, memiliki, motif, geometris, se...","[batik, betawi, memiliki, motif, geometris, se...","[batik, betawi, memiliki, motif, geometris, se...",batik betawi memiliki motif geometris sederhan...
2,Batik Celup,Batik Celup memiliki pola abstrak dengan grada...,batik-celup,batik-celup.jpg,batik celup memiliki pola abstrak dengan grada...,batik celup memiliki pola abstrak dengan grada...,"[batik, celup, memiliki, pola, abstrak, dengan...","[batik, celup, memiliki, pola, abstrak, gradas...","[batik, celup, memiliki, pola, abstrak, gradas...",batik celup memiliki pola abstrak gradasi warn...
3,Batik Cendrawasih,Batik Cendrawasih menampilkan motif burung cen...,batik-cendrawasih,batik-cendrawasih.jpg,batik cendrawasih menampilkan motif burung cen...,batik cendrawasih menampilkan motif burung cen...,"[batik, cendrawasih, menampilkan, motif, burun...","[batik, cendrawasih, menampilkan, motif, burun...","[batik, cendrawasih, menampilkan, motif, burun...",batik cendrawasih menampilkan motif burung cen...
4,Batik Ceplok,Batik Ceplok memiliki pola geometris berulang ...,batik-ceplok,batik-ceplok.jpg,batik ceplok memiliki pola geometris berulang ...,batik ceplok memiliki pola geometris berulang ...,"[batik, ceplok, memiliki, pola, geometris, ber...","[batik, ceplok, memiliki, pola, geometris, ber...","[batik, ceplok, memiliki, pola, geometris, ber...",batik ceplok memiliki pola geometris berulang ...
5,Batik Ciamis,Batik Ciamis memiliki motif sederhana dengan p...,batik-ciamis,batik-ciamis.jpg,batik ciamis memiliki motif sederhana dengan p...,batik ciamis memiliki motif sederhana dengan p...,"[batik, ciamis, memiliki, motif, sederhana, de...","[batik, ciamis, memiliki, motif, sederhana, po...","[batik, ciamis, memiliki, motif, sederhana, po...",batik ciamis memiliki motif sederhana pola bun...
6,Batik Garutan,"Batik Garutan memiliki motif bunga, daun, atau...",batik-garutan,batik-garutan.jpg,"batik garutan memiliki motif bunga, daun, atau...",batik garutan memiliki motif bunga daun atau b...,"[batik, garutan, memiliki, motif, bunga, daun,...","[batik, garutan, memiliki, motif, bunga, daun,...","[batik, garutan, memiliki, motif, bunga, daun,...",batik garutan memiliki motif bunga daun burung...
7,Batik Gentongan,Batik Gentongan memiliki motif bunga besar den...,batik-gentongan,batik-gentongan.jpg,batik gentongan memiliki motif bunga besar den...,batik gentongan memiliki motif bunga besar den...,"[batik, gentongan, memiliki, motif, bunga, bes...","[batik, gentongan, memiliki, motif, bunga, war...","[batik, gentongan, memiliki, motif, bunga, war...",batik gentongan memiliki motif bunga warna men...
8,Batik Kawung,Batik Kawung memiliki motif lingkaran geometri...,batik-kawung,batik-kawung.jpg,batik kawung memiliki motif lingkaran geometri...,batik kawung memiliki motif lingkaran geometri...,"[batik, kawung, memiliki, motif, lingkaran, ge...","[batik, kawung, memiliki, motif, lingkaran, ge...","[batik, kawung, memiliki, motif, lingkaran, ge...",batik kawung memiliki motif lingkaran geometri...
9,Batik Keraton,Batik Keraton memiliki motif klasik seperti Pa...,batik-keraton,batik-keraton.jpg,batik keraton memiliki motif klasik seperti pa...,batik keraton memiliki motif klasik seperti pa...,"[batik, keraton, memiliki, motif, klasik, se

## Save Dataset Text Pre-processing

In [29]:
df.to_csv('../../datasets/in-procces/data_text_batik_processed.csv', index=False)

# Feature Extraction

## Load Tokenizer & Model

In [30]:
tokenizer = AutoTokenizer.from_pretrained("indobenchmark/indobert-base-p2")
model = AutoModel.from_pretrained("indobenchmark/indobert-base-p2")
model.eval()

2025-02-21 06:35:39.375954: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(50000, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [31]:
projection = nn.Linear(768, 256)
projection.eval()

Linear(in_features=768, out_features=256, bias=True)

In [32]:
def get_text_embedding(text, model, tokenizer, projection, max_length=256):
    # Tangani kasus teks kosong
    if not text or str(text).strip() == "":
        return np.zeros(256)
    
    # Tokenisasi dengan padding dan truncation
    inputs = tokenizer(
        text,
        return_tensors="pt",
        max_length=max_length,
        truncation=True,
        padding="max_length"
    )
    
    with torch.no_grad():
        outputs = model(**inputs)
        # Ambil hidden state terakhir
        last_hidden_state = outputs.last_hidden_state  # [1, seq_len, 768]
        # Lakukan average pooling untuk mendapatkan vektor [768]
        embedding = last_hidden_state.mean(dim=1)       # [1, 768]
        # Proyeksikan ke 256 dimensi
        projected_embedding = projection(embedding)     # [1, 256]
        # Detach dan konversi ke NumPy
        projected_embedding = projected_embedding.squeeze().detach().numpy()
        
    return projected_embedding

## Import Preprocessed Dataset

In [33]:
df_fe = pd.read_csv('../../datasets/in-procces/data_text_batik_processed.csv')

In [34]:
df_fe.iloc[11]['preprocessing']

'batik megamendung memiliki motif awan bergradasi warna biru putih merah melambangkan kesejukan ketenangan batik warisan budaya cirebon jawa barat'

## Embedding Extraction

In [35]:
df_fe['preprocessing_embedding'] = df_fe['preprocessing'].apply(lambda x: get_text_embedding(str(x), model, tokenizer, projection).tolist())

In [36]:
np.vstack(df_fe['preprocessing_embedding']).shape


(20, 256)

## Save Embedding Extraction

In [38]:
df_fe.to_csv('../../datasets/in-procces/data_text_batik_embed.csv', index=False)

## Save Model with Projection

In [39]:
checkpoint = {
    'bert_model': model.state_dict(),
    'projection': projection.state_dict()
}

In [40]:
torch.save(checkpoint, "../../models/indobert-p2-projection256.pth")

In [41]:
checkpoint = torch.load('../../models/indobert-p2-projection256.pth')

model.load_state_dict(checkpoint['bert_model'])
projection.load_state_dict(checkpoint['projection'])

model.eval()
projection.eval()

Linear(in_features=768, out_features=256, bias=True)

# Visualization

In [53]:
df_vi = pd.read_csv('../../datasets/in-procces/data_text_batik_embed.csv')

In [54]:
df_vi

,Nama,Deskripsi,Path Folder,Path Gambar,Deskripsi_casefolding,Deskripsi_clean,tokenized,stop_removed,lemmatized,preprocessing,preprocessing_embedding
0,Batik Bali,Batik Bali memiliki motif rumit yang terinspir...,batik-bali,batik-bali.jpg,batik bali memiliki motif rumit yang terinspir...,batik bali memiliki motif rumit yang terinspir...,"['batik', 'bali', 'memiliki', 'motif', 'rumit'...","['batik', 'bali', 'memiliki', 'motif', 'rumit'...","['batik', 'bali', 'memiliki', 'motif', 'rumit'...",batik bali memiliki motif rumit terinspirasi b...,"[0.3322688043117523, 0.2934962511062622, 0.148..."
1,Batik Betawi,Batik Betawi memiliki motif geometris sederhan...,batik-betawi,batik-betawi.jpg,batik betawi memiliki motif geometris sederhan...,batik betawi memiliki motif geometris sederhan...,"['batik', 'betawi', 'memiliki', 'motif', 'geom...","['batik', 'betawi', 'memiliki', 'motif', 'geom...","['batik', 'betawi', 'memiliki', 'motif', 'geom...",batik betawi memiliki motif geometris sederhan...,"[0.07265201956033707, 0.2657928168773651, 0.19..."
2,Batik Celup,Batik Celup memiliki pola abstrak dengan grada...,batik-celup,batik-celup.jpg,batik celup memiliki pola abstrak dengan grada...,batik celup memiliki pola abstrak dengan grada...,"['batik', 'celup', 'memiliki', 'pola', 'abstra...","['batik', 'celup', 'memiliki', 'pola', 'abstra...","['batik', 'celup', 'memiliki', 'pola', 'abstra...",batik celup memiliki pola abstrak gradasi warn...,"[0.013769574463367462, 0.3355376422405243, 0.0..."
3,Batik Cendrawasih,Batik Cendrawasih menampilkan motif burung cen...,batik-cendrawasih,batik-cendrawasih.jpg,batik cendrawasih menampilkan motif burung cen...,batik cendrawasih menampilkan motif burung cen...,"['batik', 'cendrawasih', 'menampilkan', 'motif...","['batik', 'cendrawasih', 'menampilkan', 'motif...","['batik', 'cendrawasih', 'menampilkan', 'motif...",batik cendrawasih menampilkan motif burung cen...,"[0.7157406210899353, 0.7801889181137085, -0.01..."
4,Batik Ceplok,Batik Ceplok memiliki pola geometris berulang ...,batik-ceplok,batik-ceplok.jpg,batik ceplok memiliki pola geometris berulang ...,batik ceplok memiliki pola geometris berulang ...,"['batik', 'ceplok', 'memiliki', 'pola', 'geome...","['batik', 'ceplok', 'memiliki', 'pola', 'geome...","['batik', 'ceplok', 'memiliki', 'pola', 'geome...",batik ceplok memiliki pola geometris berulang ...,"[-0.05024304240942001, 0.3212023377418518, 0.0..."
5,Batik Ciamis,Batik Ciamis memiliki motif sederhana dengan p...,batik-ciamis,batik-ciamis.jpg,batik ciamis memiliki motif sederhana dengan p...,batik ciamis memiliki motif sederhana dengan p...,"['batik', 'ciamis', 'memiliki', 'motif', 'sede...","['batik', 'ciamis', 'memiliki', 'motif', 'sede...","['batik', 'ciamis', 'memiliki', 'motif', 'sede...",batik ciamis memiliki motif sederhana pola bun...,"[0.01742059737443924, 0.254853755235672, -0.23..."
6,Batik Garutan,"Batik Garutan memiliki motif bunga, daun, atau...",batik-garutan,batik-garutan.jpg,"batik garutan memiliki motif bunga, daun, atau...",batik garutan memiliki motif bunga daun atau b...,"['batik', 'garutan', 'memiliki', 'motif', 'bun...","['batik', 'garutan', 'memiliki', 'motif', 'bun...","['batik', 'garutan', 'memiliki', 'motif', 'bun...",batik garutan memiliki motif bunga daun burung...,"[0.15656530857086182, 0.6932697296142578, -0.3..."
7,Batik Gentongan,Batik Gentongan memiliki motif bunga besar den...,batik-gentongan,batik-gentongan.jpg,batik gentongan memiliki motif bunga besar den...,batik gentongan memiliki motif bunga besar den...,"['batik', 'gentongan', 'memiliki', 'motif', 'b...","['batik', 'gentongan', 'memiliki', 'motif', 'b...","['batik', 'gentongan', 'memiliki', 'motif', 'b...",batik gentongan memiliki motif bunga warna men...,"[0.34450066089630127, 0.3650816082954407, 0.04..."
8,Batik Kawung,Batik Kawung memiliki motif lingkaran geometri...,batik-kawung,batik-kawung.jpg,batik kawung memiliki motif lingkaran geometri...,batik kawung memiliki motif lingkaran geometri...,"['batik'

In [46]:
def parse_embedding(x):
    return np.array(ast.literal_eval(x))

In [86]:
embeddings = df_vi['preprocessing_embedding'].apply(parse_embedding)
embeddings_array = np.vstack(embeddings.values)

In [87]:
pca_2d = PCA(n_components=2)
embeddings_2d = pca_2d.fit_transform(embeddings_array)

In [89]:
df_vi['PCA1'] = embeddings_2d[:, 0]
df_vi['PCA2'] = embeddings_2d[:, 1]

# Buat visualisasi dengan Plotly Express
fig = px.scatter(
    df_vi, 
    x='PCA1', 
    y='PCA2', 
    text='Nama',  # Label setiap titik dengan nama batik
    color='Nama',  # Warna berbeda untuk setiap jenis batik
    title="Visualisasi 2D Embedding dengan Label Nama Batik (PCA)",
    labels={'PCA1': "Komponen Utama 1", 'PCA2': "Komponen Utama 2"},
    width=900,  # Lebar plot
    height=700  # Tinggi plot
)

# Tambahkan label titik
fig.update_traces(textposition='top center', marker=dict(size=12, opacity=0.8, line=dict(width=1, color='DarkSlateGrey')))

# Tampilkan plot interaktif
fig.show()